#**Análisis y Selección de Características para la Predicción de la Calidad del Vino mediante Regresión Lineal**
###Regina González

Se importaron los datos del archivo “Vino_Tinto.csv” al entorno de trabajo y se verificaron sus dimensiones para conocer la cantidad de filas y columnas. También se imprimieron las primeras cinco filas del DataFrame para tener una vista general de su estructura y contenido.

In [18]:
import pandas as pd
import numpy as np
data = pd.read_csv('Vino_Tinto.csv')
print(f"✅ Datos cargados: {data.shape[0]} filas, {data.shape[1]} columnas.\n")
print("🔍 Primeras 15 filas:")
display(data.head(5))

✅ Datos cargados: 1599 filas, 12 columnas.

🔍 Primeras 15 filas:


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Se dividió el DataFrame en conjuntos de entrenamiento y prueba con una proporción 80/20, asegurando que la partición fuera aleatoria para evitar sesgos. Luego, se imprimieron las dimensiones de ambos conjuntos para confirmar que la suma de sus observaciones coincidiera con el total de datos originales.

In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

print(f"📊 Dimensiones del conjunto de entrenamiento: {train.shape}")
print(f"📊 Dimensiones del conjunto de validación: {test.shape}")

if train.shape[0] + test.shape[0] == data.shape[0]:
    print("✅ La división de datos es correcta: la suma coincide con el total de observaciones.")
else:
    print("⚠️ Advertencia: La suma de los datos de entrenamiento y validación no coincide con el total de observaciones.")

📊 Dimensiones del conjunto de entrenamiento: (1279, 12)
📊 Dimensiones del conjunto de validación: (320, 12)
✅ La división de datos es correcta: la suma coincide con el total de observaciones.


Se utilizó la librería `SequentialFeatureSelector` de `mlxtend.feature_selection` para realizar una selección hacia adelante de características, con un modelo de regresión lineal como estimador. El parámetro `k_features` se configuró como un rango de (2,8), lo que permitió que el algoritmo seleccionara entre 2 y 8 características. Se configuró la selección como hacia adelante (`forward=True`), y la métrica de evaluación fue la R cuadrada (`scoring='r2'`). Para la validación cruzada, se especificó que se realizaran 10 pliegues (`cv=10`). Luego, se entrenó el modelo y se extrajeron los índices y nombres de las características seleccionadas, que se imprimieron en consola.

In [20]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

train_x = train.drop('calidad', axis=1)
train_y = train['calidad']

est = LinearRegression()

# Aplicar selección de características hacia adelante
sfs = SFS(est,
          k_features=(2, 8),
          forward=True,
          scoring='r2',
          cv=10)

# Ajustar el modelo a los datos
sfs= sfs.fit(train_x, train_y)

# Obtener los índices de las características seleccionadas
selected_features_idx = list(sfs.k_feature_idx_)

# Obtener los nombres de las características seleccionadas
selected_features_names = list(sfs.k_feature_names_)

# Imprimir los resultados
print("Índices de las características seleccionadas:", selected_features_idx)
print("Nombres de las características seleccionadas:", selected_features_names)


Índices de las características seleccionadas: [1, 4, 5, 6, 8, 9, 10]
Nombres de las características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


Se entrenó un modelo de regresión lineal utilizando únicamente las características seleccionadas en el paso anterior. Para ello, se extrajeron las columnas correspondientes de los datos de entrenamiento, asegurándose de no incluir variables no seleccionadas. Luego, el modelo se entrenó con esas variables mediante la función `fit`. Posteriormente, se hicieron predicciones sobre el conjunto de prueba, donde también se filtraron las características seleccionadas. Finalmente, se utilizó la función `r2_score` de `sklearn.metrics` para medir la capacidad de predicción del modelo, calculando y mostrando el valor de R cuadrada en consola.

In [21]:
test_x = test.drop('calidad', axis=1)
test_y = test['calidad']

# Filtrar las variables seleccionadas en los datos de entrenamiento y prueba
train_x_selected = train_x[list(sfs.k_feature_names_)]
test_x_selected = test_x[list(sfs.k_feature_names_)]

# Entrenar el modelo con las variables seleccionadas
est.fit(train_x_selected, train_y)

# Hacer predicciones en los datos de prueba
y_pred = est.predict(test_x_selected)

r2_forward = r2_score(test_y, y_pred)
print("R² del modelo con selección hacia adelante:", r2_forward)

R² del modelo con selección hacia adelante: 0.4012628835440285


Se llevó a cabo un proceso de selección hacia atrás utilizando las variables previamente seleccionadas en el paso de selección hacia adelante. Para ello, se aplicó la función `SequentialFeatureSelector` de la librería `mlxtend.feature_selection`, pero esta vez con el parámetro `forward=False`, lo que indica que se debía realizar la selección hacia atrás. Además, se definió el parámetro `k_features=(2, 5)` para restringir la cantidad de variables seleccionadas a un rango entre 2 y 5. Una vez ajustado el modelo con estas características, se obtuvieron tanto los índices como los nombres de las variables seleccionadas. Finalmente, se imprimieron en consola los resultados obtenidos, mostrando qué variables fueron elegidas tras la selección hacia atrás.

In [22]:
sfs_b = SFS(est,
          k_features=(2, 5),
          forward=False,
          scoring='r2',
          cv=10)

# Ajustar el modelo a los datos
sfs_b.fit(train_x, train_y)

# Obtener los índices de las características seleccionadas
selected_features_idx_b = list(sfs_b.k_feature_idx_)

# Obtener los nombres de las características seleccionadas
selected_features_names_b = list(sfs_b.k_feature_names_)

# Imprimir los resultados
print("Índices de las características seleccionadas:", selected_features_idx_b)
print("Nombres de las características seleccionadas:", selected_features_names_b)

Índices de las características seleccionadas: [1, 4, 6, 9, 10]
Nombres de las características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


Se repitió el proceso de entrenamiento y evaluación para el modelo utilizando únicamente las variables seleccionadas en el paso de selección hacia atrás. Se ajustó el modelo de regresión lineal a los datos de entrenamiento con estas variables seleccionadas y se realizó la predicción en los datos de prueba. Posteriormente, se calculó el valor de R cuadrada para medir la capacidad predictiva del modelo.

In [23]:
# Filtrar las variables seleccionadas en los datos de entrenamiento y prueba
train_x_selected_b = train_x[list(sfs_b.k_feature_names_)]
test_x_selected_b = test_x[list(sfs_b.k_feature_names_)]

# Entrenar el modelo con las variables seleccionadas
est.fit(train_x_selected_b, train_y)

# Hacer predicciones en los datos de prueba
y_pred_b = est.predict(test_x_selected_b)

# Calcular R² para este modelo
r2_backward = r2_score(test_y, y_pred_b)
print("R² del modelo con selección hacia atrás:", r2_backward)

R² del modelo con selección hacia atrás: 0.3958889666765396


In [24]:
print("\n--- Comparación de Modelos ---")
print(f"R² del modelo con selección hacia adelante: {r2_forward}")
print(f"R² del modelo con selección hacia atrás: {r2_backward}")

if r2_backward > r2_forward:
    print("El modelo con selección hacia atrás tiene un mejor R², lo que sugiere que eliminar algunas variables mejoró la predicción.")
elif r2_backward < r2_forward:
    print("El modelo con selección hacia adelante tiene un mejor R², lo que sugiere que incluir más variables ayudó a la predicción en este caso.")
else:
    print("Ambos modelos tienen la misma capacidad predictiva, por lo que eliminar variables no afectó el desempeño.")



--- Comparación de Modelos ---
R² del modelo con selección hacia adelante: 0.4012628835440285
R² del modelo con selección hacia atrás: 0.3958889666765396
El modelo con selección hacia adelante tiene un mejor R², lo que sugiere que incluir más variables ayudó a la predicción en este caso.


Al analizar ambos modelos, se observa que la diferencia en los valores de R² es muy pequeña, lo que sugiere que la capacidad predictiva no cambia considerablemente al realizar la selección hacia adelante o hacia atrás. Aunque el modelo con selección hacia adelante incluye más variables, lo que podría parecer ventajoso a primera vista, el modelo con selección hacia atrás elimina algunas variables innecesarias, lo que lo hace más eficiente.

Dado que ambos modelos logran resultados similares en términos de R², creo que el modelo con selección hacia atrás sería la mejor opción. Esto se debe a que, al reducir la cantidad de variables, se optimiza el proceso, haciéndolo más rápido y menos pesado en términos computacionales. Además, al eliminar variables que no aportan mucho a la predicción, el modelo tiende a generalizar mejor, lo que podría reducir el riesgo de sobreajuste.